In [1]:
from music21 import converter , instrument , note , chord
import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout , LSTM , Activation
from keras.utils import np_utils
import os
import numpy as np

Using TensorFlow backend.


In [2]:
directory = os.path.dirname(os.path.abspath('__file__')) + '/midi_songs/'
notes = []
for file in os.listdir(directory):
    if '.mid' in file:
        print(file)
        midi = converter.parse(directory + '/' + file)
        notes_to_parse = None
        
        parts = instrument.partitionByInstrument(midi)
        
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element , note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

ff11_awakening_piano.mid
Oppressed.mid
ff6shap.mid
Zelda_Overworld.mid
FF3_Battle_(Piano).mid
Rachel_Piano_tempofix.mid
AT.mid
8.mid
ultros.mid
decisive.mid
rufus.mid
cosmo.mid
figaro.mid
Rydia_pc.mid
thoughts.mid
Fierce_Battle_(Piano).mid
lurk_in_dark.mid
DOS.mid
ahead_on_our_way_piano.mid
great_war.mid
relmstheme-piano.mid
VincentPiano.mid
costadsol.mid
ff1battp.mid
Still_Alive-1.mid
Suteki_Da_Ne_(Piano_Version).mid
bcm.mid
Ff7-One_Winged.mid
goldsaucer.mid
ultimafro.mid
dontbeafraid.mid
OTD5YA.mid
EyesOnMePiano.mid
FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Cids.mid
Ff7-Cinco.mid
traitor.mid
FFIX_Piano.mid
ff4-town.mid
Fiend_Battle_(Piano).mid
Life_Stream.mid
FFIII_Edgar_And_Sabin_Piano.mid
Final_Fantasy_7_-_Judgement_Day_Piano.mid
BlueStone_LastDungeon.mid
Ff4-BattleLust.mid
caitsith.mid
fortresscondor.mid
FFIXQuMarshP.mid
Final_Fantasy_Matouyas_Cave_Piano.mid
redwings.mid
sobf.mid
tpirtsd-piano.mid
mining.mid
ViviinAlexandria.mid
Eternal_Harvest.mid
path_of_repentance.mi

In [3]:
sequence_length = 100

pitchnames = sorted(set(item for item in notes))
notes_to_int = {}

for number , note in enumerate(pitchnames):
    notes_to_int[note] = number
    
network_input , network_output = [] , []
for i in range(0 , len(notes) - sequence_length):
    seq_inp = notes[i : i+sequence_length]
    network_input.append([notes_to_int[note] for note in seq_inp])
    network_output.append(notes_to_int[notes[i + sequence_length]])
    
n_vocab = len(set(notes))
n_patterns = len(network_input)

network_input = np.reshape(network_input , (n_patterns , sequence_length , 1))
network_input = network_input/float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [4]:
network_input.shape

(55720, 100, 1)

In [5]:
model = Sequential()
model.add(LSTM(
    64,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dense(32))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 64)           16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 32)           12416     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
__________

In [7]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint] 

In [8]:
model.fit(network_input, network_output, epochs=10, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
55720/55720 [==============================] - 219s 4ms/step - loss: 4.7076
Epoch 2/10
55720/55720 [==============================] - 220s 4ms/step - loss: 4.5567
Epoch 3/10
55720/55720 [==============================] - 219s 4ms/step - loss: 4.5092
Epoch 4/10
55720/55720 [==============================] - 221s 4ms/step - loss: 4.4661
Epoch 5/10
55720/55720 [==============================] - 227s 4ms/step - loss: 4.4041
Epoch 6/10
55720/55720 [==============================] - 223s 4ms/step - loss: 4.3480
Epoch 7/10
55720/55720 [==============================] - 220s 4ms/step - loss: 4.3148
Epoch 8/10
55720/55720 [==============================] - 224s 4ms/step - loss: 4.2886
Epoch 9/10
55720/55720 [==============================] - 221s 4ms/step - loss: 4.2556
Epoch 10/10
55720/55720 [==============================] - 226s 4ms/step - loss: 4.2664


In [9]:
import pickle
filename = 'music_model.sav'
pickle.dump(model , open(filename , 'wb'))

In [11]:
model = Sequential()
model.add(LSTM(
    64,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dense(32))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.load_weights('weights-improvement-09-4.2556-bigger.hdf5')

In [17]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
for note_index in range(50):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern , index)
    pattern = pattern[1:len(pattern)]

In [20]:
offset = 0
output_notes = []
for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

In [23]:
midi_stream = stream.Stream(output_notes)

In [24]:
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'

In [22]:
#run it above the cell[16] . because note is overridden and stream is previously not imported
from music21 import converter , instrument , note , chord , stream